In [1]:
from dotenv import load_dotenv
import os
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import GPT4AllEmbeddings
import glob
from tqdm import tqdm
import pickle
import json
import numpy as np
import concurrent.futures
import pandas as pd

# load qa dataset
# Reading JSON data
json_filename = "QA_dataset.json"
with open(json_filename, 'r') as file:
    data_loaded = json.load(file)

os.environ["OPENAI_API_KEY"] = ##

def process(item):

    try:
        pdf = item["filename"]
        ques= [item["question_1"] , item["question_2"], item["question_3"]]
        anss = [item["answer_1"], item["answer_2"], item["answer_3"]]
    except:
        print("Key error, please check. ", item)
        return []
    
    pdf_reader = PdfReader(pdf)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()

    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=500,
        chunk_overlap=100,
        length_function=len
    )  

    chunks = text_splitter.split_text(text)

    # embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    embeddings = GPT4AllEmbeddings()
    save_faiss_index = False
    index_filename = "faiss_index.gpt4all"
    if save_faiss_index:
        if not os.path.exists(index_filename):
            knowledge_base = FAISS.from_texts(chunks, embeddings)
            knowledge_base.save_local(index_filename)
        else:
            knowledge_base = FAISS.load_local(index_filename, embeddings, allow_dangerous_deserialization=True)
    else:
        knowledge_base = FAISS.from_texts(chunks, embeddings)
        
    result = []
    for query, answer in zip(ques, anss):
        docs = knowledge_base.similarity_search(query)

        llm = OpenAI()
        chain = load_qa_chain(llm, chain_type="stuff")
        response = chain.run(input_documents=docs, question=query)

        print("======================")
        print(f"Query: {query}")
        print(f"Ans: {answer}")
        print(f"Res: {response}")
        
        verifier_prompt = """## Role: Answer verifier

## Goal
You can judge whether the answer is correct or not. 

## Rule
- If the key information predicted answer is same as the ground truth answer, then the answer is correct.


## Output format
{
"reason": "fill the reason why the predicted answer is wrong (False) or correct (True).", 
"answer": True or False
}

""" + f"""

The question is: {query}
Ground truth is: {answer}
Predicted answer is: {response}
"""
        verified_output = llm.invoke(verifier_prompt)
        print("verified_output: ", verified_output)
        verified_bool = verified_output.split('"answer": ')[-1]
        if "True" in verified_bool:
            verified_bool = 1
        else:
            verified_bool = 0

        result.append([pdf, query, answer, response, verified_bool])

    return result

with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    results = list(executor.map(process, data_loaded))

# save results
os.makedirs("./results", exist_ok=True)
result_csv = "./results/qa_single_cs-1000.csv"
df = pd.DataFrame(results, columns = ["filename", "query", "answer", "response", "verified_bool"])
df.to_csv(result_csv, index=False)

# calculate the accuracy
correctness = df.loc[:, "verified_bool"].values
print(correctness)
print(np.mean(correctness))

Key error, please check.  {'question_1': 'When was Huawei founded?', 'question_2': 'How much revenue did Huawei generate in 2023?', 'question_3': 'How many employees does Huawei have in 2023?', 'filename': './docs\\annual_report_2023_en.pdf', 'first_5_page_texts': 'Huawei Investment & Holding Co., Ltd.\n2023 ANNUAL REPORTWho is Huawei?\nFounded in 1987, Huawei is a leading \nglobal provider of information and \ncommunications technology (ICT) \ninfrastructure and smart devices. We \nhave approximately 207,000 employees \nand we operate in over 170 countries \nand regions, serving more than three \nbillion people around the world. We are \ncommitted to bringing digital to every \nperson, home and organization for a \nfully connected, intelligent world.Walking the walk for steady,  \nlong-term growth\nLaying the groundwork for an intelligent world\nIn 2023, we witnessed rapid advancement in emerging \ntechnologies like artificial intelligence (AI) and nonstop innovation \nin new applicat

Created a chunk of size 2612, which is longer than the specified 500
Created a chunk of size 1801, which is longer than the specified 500
Created a chunk of size 552, which is longer than the specified 500
Created a chunk of size 552, which is longer than the specified 500
Created a chunk of size 1343, which is longer than the specified 500
Created a chunk of size 650, which is longer than the specified 500
Created a chunk of size 702, which is longer than the specified 500
Created a chunk of size 686, which is longer than the specified 500
Created a chunk of size 508, which is longer than the specified 500
Created a chunk of size 566, which is longer than the specified 500
Created a chunk of size 852, which is longer than the specified 500
Created a chunk of size 862, which is longer than the specified 500
Created a chunk of size 520, which is longer than the specified 500
Created a chunk of size 578, which is longer than the specified 500
Created a chunk of size 696, which is longer 

Query: What is the per share dividend amount announced by McDonald's?
Ans: $1.67
Res:  $1.67 per share
Query: What was Apple Inc.'s net income in the twelve months ended September 30, 2023?
Ans: $96,995 million
Res:  Apple Inc.'s net income in the twelve months ended September 30, 2023 was /dollarsign 96,995 million.
verified_output:  {
"reason": "The predicted answer is the same as the ground truth answer.", 
"answer": True
}
verified_output:  
{
"reason": "The predicted answer is correct as it matches the ground truth answer of $96,995 million.", 
"answer": True
}
Query: How much did Apple Inc. spend on repurchases of common stock in the twelve months ended September 30, 2023?
Ans: $77,550 million
Res:  Apple Inc. spent $77,550 million on repurchases of common stock in the twelve months ended September 30, 2023.
verified_output:  {
"reason": "The predicted answer is correct because it states the exact amount of $77,550 million, which matches the ground truth answer.", 
"answer": True

Created a chunk of size 555, which is longer than the specified 500
Created a chunk of size 594, which is longer than the specified 500
Created a chunk of size 822, which is longer than the specified 500
Created a chunk of size 992, which is longer than the specified 500
Created a chunk of size 507, which is longer than the specified 500
Created a chunk of size 680, which is longer than the specified 500
Created a chunk of size 567, which is longer than the specified 500
Created a chunk of size 829, which is longer than the specified 500
Created a chunk of size 801, which is longer than the specified 500
Created a chunk of size 890, which is longer than the specified 500
Created a chunk of size 635, which is longer than the specified 500
Created a chunk of size 630, which is longer than the specified 500
Created a chunk of size 1430, which is longer than the specified 500
Created a chunk of size 733, which is longer than the specified 500
Created a chunk of size 1032, which is longer t